# Joke Recommendation Engine Using - Collabrative Based Filterning

# Importing Necessary Libraries

Downloading the dataset

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abhinavmoudgil95/short-jokes")

print("/kaggle/working/", path)

/kaggle/working/ /kaggle/input/short-jokes


In [3]:
import pandas as pd
import numpy as np

# Reading the Dataset

In [4]:
jokes = pd.read_csv("/kaggle/input/jester-collaborative-filtering-dataset/JokeText.csv")
print(jokes.head())
print(jokes.shape)
print(jokes.info())

   JokeId                                           JokeText
0       0  A man visits the doctor. The doctor says "I ha...
1       1  This couple had an excellent relationship goin...
2       2  Q. What's 200 feet long and has 4 teeth? \n\nA...
3       3  Q. What's the difference between a man and a t...
4       4  Q.\tWhat's O. J. Simpson's Internet address? \...
(100, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   JokeId    100 non-null    int64 
 1   JokeText  100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB
None


In [5]:
jokes['JokeText'].iloc[0]

'A man visits the doctor. The doctor says "I have bad news for you.You have\ncancer and Alzheimer\'s disease". \nThe man replies "Well,thank God I don\'t have cancer!"\n'

In [6]:
review1 = pd.read_csv("/kaggle/input/jester-collaborative-filtering-dataset/UserRatings1.csv")
print(review1.head())
print(review1.shape)
print(review1.info())

   JokeId  User1  User2  User3  User4  User5  User6  User7  User8  User9  ...  \
0       0   5.10  -8.79  -3.50   7.14  -8.79   9.22  -4.03   3.11  -3.64  ...   
1       1   4.90  -0.87  -2.91  -3.88  -0.58   9.37  -1.55   0.92  -3.35  ...   
2       2   1.75   1.99  -2.18  -3.06  -0.58  -3.93  -3.64   7.52  -6.46  ...   
3       3  -4.17  -4.61  -0.10   0.05   8.98   9.27  -6.99   0.49  -3.40  ...   
4       4   5.15   5.39   7.52   6.26   7.67   3.45   5.44  -0.58   1.26  ...   

   User36701  User36702  User36703  User36704  User36705  User36706  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN      -5.63        NaN      -6.07   
2        NaN        NaN        NaN        NaN        NaN       4.08   
3        NaN        NaN        NaN        NaN        NaN        NaN   
4       2.28      -0.49        5.1      -0.29      -3.54      -1.36   

   User36707  User36708  User36709  User36710  
0        NaN        NaN       2.91    

In [7]:
review2 = pd.read_csv("/kaggle/input/jester-collaborative-filtering-dataset/UserRatings2.csv")
print(review2.head())
print(review2.shape)
print(review2.info())

   JokeId  User36711  User36712  User36713  User36714  User36715  User36716  \
0       0        NaN        NaN        NaN       3.93        NaN        NaN   
1       1        NaN        NaN        NaN        NaN        NaN        NaN   
2       2        NaN        NaN        NaN        NaN        NaN        NaN   
3       3        NaN        NaN        NaN        NaN        NaN        NaN   
4       4       5.68       1.07       8.11      -2.33      -5.83        8.2   

   User36717  User36718  User36719  ...  User73412  User73413  User73414  \
0        NaN        NaN        NaN  ...        NaN        NaN        NaN   
1        NaN        NaN       4.81  ...        NaN        NaN        NaN   
2        NaN        NaN        NaN  ...        NaN        NaN        NaN   
3        NaN        NaN        NaN  ...        NaN        NaN        NaN   
4      -5.83       1.94       0.10  ...       3.64       4.32       6.99   

   User73415  User73416  User73417  User73418  User73419  User73420 

# Data Preprocessing

In [8]:
#filling the NAN values with 0
review1=review1.fillna(0)
review1.head(3)

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User36701,User36702,User36703,User36704,User36705,User36706,User36707,User36708,User36709,User36710
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,2.91,0.0
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,0.0,0.0,0.0,-5.63,0.0,-6.07,0.0,-1.6,-4.56,0.0
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,0.0,0.0,0.0,0.00,0.0,4.08,0.0,0.0,8.98,0.0


In [9]:
review2=review2.fillna(0)
review2.head(3)

,JokeId,User36711,User36712,User36713,User36714,User36715,User36716,User36717,User36718,User36719,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
0,0,0.0,0.0,0.0,3.93,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,4.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Combine them into 1

In [10]:
reviews = pd.merge(review1, review2, on='JokeId', how='left')

In [11]:
reviews.head()

,JokeId,User1,User2,User3,User4,User5,User6,User7,User8,User9,...,User73412,User73413,User73414,User73415,User73416,User73417,User73418,User73419,User73420,User73421
0,0,5.10,-8.79,-3.50,7.14,-8.79,9.22,-4.03,3.11,-3.64,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0
1,1,4.90,-0.87,-2.91,-3.88,-0.58,9.37,-1.55,0.92,-3.35,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0
2,2,1.75,1.99,-2.18,-3.06,-0.58,-3.93,-3.64,7.52,-6.46,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0
3,3,-4.17,-4.61,-0.10,0.05,8.98,9.27,-6.99,0.49,-3.40,...,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0
4,4,5.15,5.39,7.52,6.26,7.67,3.45,5.44,-0.58,1.26,...,3.64,4.32,6.99,-9.66,-8.4,-0.63,9.51,-7.67,-1.6,8.3


In [12]:
reviews['User1'].describe()

count    100.000000
mean       1.747000
std        4.773929
min       -9.850000
25%       -1.105000
50%        3.130000
75%        4.912500
max        9.320000
Name: User1, dtype: float64

# Model Building

In [13]:
import random
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

In [15]:
# Reshape from wide to long format
ratings = reviews.melt(id_vars=['JokeId'], 
                       var_name='user_id', 
                       value_name='rating')

In [18]:
# Reset the index
ratings.reset_index(drop=True, inplace=True)

In [19]:
ratings.head()

,JokeId,user_id,rating
0,0,User1,5.10
1,1,User1,4.90
2,2,User1,1.75
3,3,User1,-4.17
4,4,User1,5.15


In [20]:
ratings.shape

(7342100, 3)

In [23]:
# Initialize the dataset for Surprise
reader = Reader(rating_scale=(-10, 10))  
data = Dataset.load_from_df(ratings[['user_id', 'JokeId', 'rating']], reader)

model = SVD()
# Train the model
trainset = data.build_full_trainset()
model.fit(trainset)

In [24]:
jokes

,JokeId,JokeText
0,0,"A man visits the doctor. The doctor says ""I ha..."
1,1,This couple had an excellent relationship goin...
2,2,Q. What's 200 feet long and has 4 teeth? \n\nA...
3,3,Q. What's the difference between a man and a t...
4,4,Q.\tWhat's O. J. Simpson's Internet address? \...
...,...,...
95,95,Two attorneys went into a diner and ordered tw...
96,96,A teacher is explaining to her class how diffe...
97,97,Age and Womanhood\n\n1. Between the ages of 13...
98,98,A bus station is where a bus stops.\nA train s...


In [25]:
# Random joke for user
random_joke_id = random.randint(0, 99)
print("Here's a joke for you:")
print(jokes.loc[random_joke_id, 'JokeText'])

Here's a joke for you:
Two men are discussing the age old question: who enjoys sex more, the
man or the woman?  A woman walks by and listens in for awhile and then
interrupts: 
"Listen you guys. You know when your ear itches and you put in your 
little finger and wiggle it around for awhile?  Afterward,
which feels better, your finger or your ear?"



In [28]:
# Get user's rating for the joke
user_id = "new_user"  
rating = float(input(f"Rate this joke (0-10): "))

# Add the new rating to the ratings DataFrame
new_rating = pd.DataFrame({'user_id': [user_id], 'joke_id': [random_joke_id], 'rating': [rating]})
ratings = pd.concat([ratings, new_rating], ignore_index=True)

Rate this joke (0-10):  8


In [29]:
# Predict ratings for all jokes for the new user
predicted_ratings = {}
for joke_id in jokes['JokeId']:
    predicted_ratings[joke_id] = model.predict(user_id, joke_id).est

# Sort the jokes by predicted rating
sorted_jokes = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)

# Recommend the next joke
for joke_id, pred_rating in sorted_jokes:
    if joke_id != random_joke_id:  # Exclude the joke already rated
        print("\nYou might like this joke next:")
        print(jokes.loc[jokes['JokeId'] == joke_id, 'JokeText'].values[0])
        break


You might like this joke next:
A guy goes into confession and says to the priest, "Father, I'm 80 years
old, widower, with 11 grandchildren. Last night I met two beautiful flight
attendants. They took me home and I made love to both of them. Twice."

The priest said: "Well, my son, when was the last time you were in
confession?"
 "Never Father, I'm Jewish."
 "So then, why are you telling me?"
 "I'm telling everybody."



# Model Saving

In [30]:
import pickle

# Save the trained model to a file
with open('joke_recommendation_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved to 'joke_recommendation_model.pkl'")

Model saved to 'joke_recommendation_model.pkl'
